# Imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

C:\Users\Joshua Amresh\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Iteration 1

### Why not going with Train Test Split here?

To avoid data leakage, it is always preferable to do train test split before feature engineering and preprocessing. In case of imbalanced class, use stratified train test split. Initially, thought to go this way only but the problem is when seeing the mode imputation, in train calculating mode is okay, but no guarantee in test the frequency will be there to get the same mode. So the best way i'm seeing right now is to do feature engineering on whole data. In real world office projects, save each mode and median so that they can be imputed in hardcoded way in the deployment scenario.

In [2]:
data = pd.read_csv("./Output_CSVs/Prepped Data/merged_nulls_dropped.csv")

In [3]:
data.shape

(590540, 361)

In [4]:
X_train = data.drop(['isFraud', 'TransactionID', 'Unnamed: 0'], axis = 1)
y_train = data['isFraud']

In [6]:
X_train.shape

(590540, 358)

In [7]:
y_train.value_counts()

0    569877
1     20663
Name: isFraud, dtype: int64

## Feature Engineering

### Imputations

In [8]:
d_features = ['D1', 'D2', 'D3', 'D4', 'D5', 'D10', 'D11', 'D15']

In [9]:
# replace missing value with a new label
def replace_d_feature(dataset,d_features):
    data = dataset.copy()
    data[d_features] = data[d_features].fillna(-1000)
    return data

X_train = replace_d_feature(X_train,d_features)

X_train[d_features].isnull().sum()

D1     0
D2     0
D3     0
D4     0
D5     0
D10    0
D11    0
D15    0
dtype: int64

In [10]:
c_features = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14']

In [11]:
# replace missing value with a new label
def replace_c_feature(dataset,c_features):
    data = dataset.copy()
    data[c_features] = data[c_features].fillna(0)
    return data

X_train = replace_c_feature(X_train,c_features)

X_train[c_features].isnull().sum()

C1     0
C2     0
C3     0
C4     0
C5     0
C6     0
C7     0
C8     0
C9     0
C10    0
C11    0
C12    0
C13    0
C14    0
dtype: int64

### TransactionDT feature creation

In [12]:
X_train['TransactionDT'].isnull().sum()

0

In [13]:
X_train['TransactionDT'].min()

86400

In [14]:
X_train.shape

(590540, 358)

In [15]:
X_train['TransactionDT_derived'] = X_train['TransactionDT'] - 86400

In [16]:
X_train.shape

(590540, 359)

In [17]:
X_train = X_train.drop(['TransactionDT'], axis = 1)

In [18]:
X_train.shape

(590540, 358)

### Mode and Median imputations

In [19]:
numerical_features = [feature for feature in X_train.columns if X_train[feature].dtypes != 'O']

print('numer of numerical variables = {}'.format(len(numerical_features)))

X_train[numerical_features].head()

numer of numerical variables = 332


,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,C1,C2,...,id_01,id_02,id_05,id_06,id_11,id_13,id_17,id_19,id_20,TransactionDT_derived
0,68.5,13926,NaN,150.0,142.0,315.0,87.0,19.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,29.0,2755,404.0,150.0,102.0,325.0,87.0,NaN,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,59.0,4663,490.0,150.0,166.0,330.0,87.0,287.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,69
3,50.0,18132,567.0,150.0,117.0,476.0,87.0,NaN,2.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,99
4,50.0,4497,514.0,150.0,102.0,420.0,87.0,NaN,1.0,1.0,...,0.0,70787.0,NaN,NaN,100.0,NaN,166.0,542.0,144.0,106


In [20]:
discrete_features = [feature for feature in numerical_features if len(X_train[feature].unique()) < 25]
print("discrete var count : {}".format(len(discrete_features)))

discrete var count : 140


In [21]:
only_numerical_features = list(set(numerical_features)-set(discrete_features))

In [22]:
len(only_numerical_features)

192

In [23]:
more_discrete_features = ['V170','V181','V237','V172','V199','V246','C3','V186','V261','V190','V44','V78','V77','V299','V239',
                          'V257','V238','id_06','V200','V87','V282','V252','V287','V220','V235','V236','V45','V196','V176',
                          'V192','V188','V86','V185','V189','id_01','V193','V183','V100']

In [24]:
mode_impute_features = discrete_features+more_discrete_features

In [25]:
mode_impute_features

['V1',
 'V2',
 'V3',
 'V4',
 'V5',
 'V6',
 'V7',
 'V8',
 'V9',
 'V10',
 'V11',
 'V12',
 'V13',
 'V14',
 'V15',
 'V16',
 'V17',
 'V18',
 'V19',
 'V20',
 'V21',
 'V22',
 'V23',
 'V24',
 'V25',
 'V26',
 'V27',
 'V28',
 'V29',
 'V30',
 'V31',
 'V32',
 'V33',
 'V34',
 'V35',
 'V36',
 'V39',
 'V40',
 'V41',
 'V42',
 'V43',
 'V46',
 'V47',
 'V48',
 'V49',
 'V50',
 'V51',
 'V52',
 'V53',
 'V54',
 'V55',
 'V57',
 'V58',
 'V59',
 'V60',
 'V61',
 'V62',
 'V63',
 'V64',
 'V65',
 'V66',
 'V67',
 'V68',
 'V69',
 'V70',
 'V71',
 'V72',
 'V73',
 'V74',
 'V75',
 'V76',
 'V79',
 'V80',
 'V81',
 'V82',
 'V83',
 'V84',
 'V85',
 'V88',
 'V89',
 'V90',
 'V91',
 'V92',
 'V93',
 'V94',
 'V98',
 'V104',
 'V107',
 'V108',
 'V109',
 'V110',
 'V111',
 'V112',
 'V113',
 'V114',
 'V115',
 'V116',
 'V117',
 'V118',
 'V119',
 'V120',
 'V121',
 'V122',
 'V123',
 'V124',
 'V125',
 'V169',
 'V173',
 'V174',
 'V175',
 'V184',
 'V191',
 'V194',
 'V195',
 'V197',
 'V198',
 'V223',
 'V240',
 'V241',
 'V242',
 'V244',
 'V247

In [26]:
len(mode_impute_features)

178

In [27]:
mode_impute_features = [feature for feature in mode_impute_features if feature not in d_features+c_features+['TransactionDT_derived']]

In [28]:
len(mode_impute_features)

177

In [29]:
mode_impute_features = set(mode_impute_features)

In [30]:
len(mode_impute_features)

177

In [31]:
for feature in mode_impute_features:
    X_train[feature] = X_train[feature].fillna(X_train[feature].mode()[0])

In [32]:
X_train[mode_impute_features].isnull().sum()

V45     0
V198    0
V74     0
V35     0
V13     0
       ..
V41     0
V122    0
V123    0
V66     0
V68     0
Length: 177, dtype: int64

In [33]:
mode_impute_features = list(mode_impute_features)

In [34]:
median_impute_features = [feature for feature in only_numerical_features if feature not in d_features+mode_impute_features+c_features+['TransactionDT_derived']]

In [35]:
len(median_impute_features)

132

In [36]:
for feature in median_impute_features:
    X_train[feature] = X_train[feature].fillna(X_train[feature].median())

In [37]:
X_train[median_impute_features].isnull().sum()

V279     0
V177     0
V233     0
V128     0
card5    0
        ..
V134     0
V211     0
V290     0
V221     0
V204     0
Length: 132, dtype: int64

### Categorical Features Imputation

In [38]:
notfound_impute_features = ['id_12','id_15','id_16','id_28','id_29']

In [39]:
# replace missing value with a new label
def replace_notfound_feature(dataset,notfound_impute_features):
    data = dataset.copy()
    data[notfound_impute_features] = data[notfound_impute_features].fillna('NotFound')
    return data

X_train = replace_notfound_feature(X_train,notfound_impute_features)

X_train[notfound_impute_features].isnull().sum()

id_12    0
id_15    0
id_16    0
id_28    0
id_29    0
dtype: int64

In [40]:
missing_impute_features = ['id_31','id_35','id_36','id_37','id_38','M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9',
                           'DeviceType', 'DeviceInfo', 'ProductCD','card4', 'card6','P_emaildomain','R_emaildomain']

In [41]:
# replace missing value with a new label
def replace_missing_feature(dataset,missing_impute_features):
    data = dataset.copy()
    data[missing_impute_features] = data[missing_impute_features].fillna('Missing')
    return data

X_train = replace_missing_feature(X_train,missing_impute_features)

X_train[missing_impute_features].isnull().sum()

id_31            0
id_35            0
id_36            0
id_37            0
id_38            0
M1               0
M2               0
M3               0
M4               0
M5               0
M6               0
M7               0
M8               0
M9               0
DeviceType       0
DeviceInfo       0
ProductCD        0
card4            0
card6            0
P_emaildomain    0
R_emaildomain    0
dtype: int64

### Scaling, Onehot-encoding, Binning

Not trying binning for now as going forward with tree based model

In [102]:
id_31_df = pd.DataFrame(X_train['id_31'].value_counts())

In [103]:
device_df = pd.DataFrame(X_train['DeviceInfo'].value_counts())

In [104]:
p_mail_df = pd.DataFrame(X_train['P_emaildomain'].value_counts())

In [105]:
r_mail_df = pd.DataFrame(X_train['R_emaildomain'].value_counts())

In [106]:
id_31_df.to_csv("./Output_CSVs/Analysis Outputs/id_31.csv")

In [107]:
device_df.to_csv("./Output_CSVs/Analysis Outputs/device_df.csv")
p_mail_df.to_csv("./Output_CSVs/Analysis Outputs/p_mail_df.csv")
r_mail_df.to_csv("./Output_CSVs/Analysis Outputs/r_mail_df.csv")

In [42]:
replace_id_31 = {'id_31' : {'Missing':'Missing',
'chrome 63.0':'chrome 63.0',
'mobile safari 11.0':'mobile safari 11.0',
'mobile safari generic':'mobile safari generic',
'ie 11.0 for desktop':'ie 11.0 for desktop',
'safari generic':'safari generic',
'chrome 62.0':'chrome 62.0',
'chrome 65.0':'chrome 65.0',
'chrome 64.0':'chrome 64.0',
'chrome 63.0 for android':'chrome 63.0 for android',
'chrome generic':'chrome generic',
'chrome 66.0':'chrome 66.0',
'edge 16.0':'edge 16.0',
'chrome 64.0 for android':'chrome 64.0 for android',
'firefox 57.0':'firefox 57.0',
'chrome 65.0 for android':'chrome 65.0 for android',
'mobile safari 10.0':'mobile safari 10.0',
'chrome 66.0 for android':'chrome 66.0 for android',
'chrome 62.0 for android':'chrome 62.0 for android',
'edge 15.0':'edge 15.0',
'chrome generic for android':'Others',
'firefox 59.0':'Others',
'samsung browser 6.2':'Others',
'firefox 58.0':'Others',
'chrome 49.0':'Others',
'ie 11.0 for tablet':'Others',
'firefox':'Others',
'chrome 61.0':'Others',
'safari 11.0':'Others',
'mobile safari 9.0':'Others',
'chrome 61.0 for android':'Others',
'samsung browser 6.4':'Others',
'edge 14.0':'Others',
'chrome':'Others',
'firefox 52.0':'Others',
'chrome 60.0':'Others',
'other':'Others',
'chrome 60.0 for android':'Others',
'chrome 55.0 for android':'Others',
'android webview 4.0':'Others',
'chrome 58.0 for android':'Others',
'chrome 63.0 for ios':'Others',
'chrome 56.0 for android':'Others',
'firefox 60.0':'Others',
'samsung browser generic':'Others',
'chrome 59.0 for android':'Others',
'firefox 48.0':'Others',
'chrome 58.0':'Others',
'Samsung/SM-G532M':'Others',
'chrome 66.0 for ios':'Others',
'chrome 50.0 for android':'Others',
'chrome 52.0 for android':'Others',
'chrome 59.0':'Others',
'firefox 56.0':'Others',
'opera 49.0':'Others',
'safari 10.0':'Others',
'samsung browser 7.0':'Others',
'android browser 4.0':'Others',
'firefox generic':'Others',
'chrome 56.0':'Others',
'chrome 57.0':'Others',
'mobile safari uiwebview':'Others',
'chrome 57.0 for android':'Others',
'Generic/Android 7.0':'Others',
'chrome 54.0 for android':'Others',
'chrome 43.0 for android':'Others',
'chrome 65.0 for ios':'Others',
'opera generic':'Others',
'edge 17.0':'Others',
'chrome 55.0':'Others',
'opera':'Others',
'chrome 46.0 for android':'Others',
'chrome 62.0 for ios':'Others',
'google search application 49.0':'Others',
'ie':'Others',
'edge':'Others',
'Samsung/SM-G531H':'Others',
'chrome 51.0':'Others',
'opera 52.0':'Others',
'edge 13.0':'Others',
'Generic/Android':'Others',
'google search application 48.0':'Others',
'mobile safari 8.0':'Others',
'opera 53.0':'Others',
'chrome 64.0 for ios':'Others',
'chrome 49.0 for android':'Others',
'opera 51.0':'Others',
'samsung browser 5.4':'Others',
'chrome 51.0 for android':'Others',
'chrome 53.0 for android':'Others',
'samsung browser 4.0':'Others',
'safari 9.0':'Others',
'firefox 47.0':'Others',
'chrome 67.0':'Others',
'google':'Others',
'samsung browser 5.2':'Others',
'Microsoft/Windows':'Others',
'firefox 55.0':'Others',
'silk':'Others',
'samsung browser 4.2':'Others',
'samsung':'Others',
'safari':'Others',
'ZTE/Blade':'Others',
'mobile':'Others',
'line':'Others',
'aol':'Others',
'maxthon':'Others',
'samsung browser 3.3':'Others',
'icedragon':'Others',
'comodo':'Others',
'Lanix/Ilium':'Others',
'Mozilla/Firefox':'Others',
'palemoon':'Others',
'puffin':'Others',
'Cherry':'Others',
'chromium':'Others',
'waterfox':'Others',
'Inco/Minion':'Others',
'seamonkey':'Others',
'iron':'Others',
'chrome 67.0 for android':'Others',
'BLU/Dash':'Others',
'chrome 69.0':'Others',
'Nokia/Lumia':'Others',
'Samsung/SCH':'Others',
'facebook':'Others'}}

In [43]:
X_train.replace(replace_id_31, inplace=True)

In [49]:
id_31_fixed_df = pd.DataFrame(X_train['id_31'].value_counts())

In [50]:
id_31_fixed_df.to_csv("./Output_CSVs/Analysis Outputs/id_31_fixed.csv")

In [44]:
replace_p_email = {'P_emaildomain':{'gmail.com':'gmail.com',
'yahoo.com':'yahoo.com',
'Missing':'Missing',
'hotmail.com':'hotmail.com',
'anonymous.com':'anonymous.com',
'aol.com':'aol.com',
'comcast.net':'comcast.net',
'icloud.com':'icloud.com',
'outlook.com':'outlook.com',
'msn.com':'msn.com',
'att.net':'att.net',
'live.com':'live.com',
'sbcglobal.net':'sbcglobal.net',
'verizon.net':'verizon.net',
'ymail.com':'ymail.com',
'bellsouth.net':'bellsouth.net',
'yahoo.com.mx':'yahoo.com.mx',
'me.com':'me.com',
'cox.net':'Others',
'optonline.net':'Others',
'charter.net':'Others',
'live.com.mx':'Others',
'rocketmail.com':'Others',
'mail.com':'Others',
'earthlink.net':'Others',
'gmail':'Others',
'mac.com':'Others',
'outlook.es':'Others',
'juno.com':'Others',
'windstream.net':'Others',
'aim.com':'Others',
'roadrunner.com':'Others',
'hotmail.es':'Others',
'hotmail.fr':'Others',
'frontier.com':'Others',
'embarqmail.com':'Others',
'netzero.com':'Others',
'web.de':'Others',
'twc.com':'Others',
'prodigy.net.mx':'Others',
'frontiernet.net':'Others',
'q.com':'Others',
'centurylink.net':'Others',
'suddenlink.net':'Others',
'sc.rr.com':'Others',
'netzero.net':'Others',
'cfl.rr.com':'Others',
'gmx.de':'Others',
'yahoo.fr':'Others',
'cableone.net':'Others',
'yahoo.es':'Others',
'hotmail.co.uk':'Others',
'yahoo.de':'Others',
'protonmail.com':'Others',
'ptd.net':'Others',
'live.fr':'Others',
'yahoo.co.uk':'Others',
'hotmail.de':'Others',
'servicios-ta.com':'Others',
'yahoo.co.jp':'Others'}}

In [45]:
replace_r_email = {'R_emaildomain':{'Missing':'Missing',
'gmail.com':'gmail.com',
'hotmail.com':'hotmail.com',
'anonymous.com':'anonymous.com',
'yahoo.com':'yahoo.com',
'aol.com':'aol.com',
'outlook.com':'outlook.com',
'comcast.net':'comcast.net',
'yahoo.com.mx':'yahoo.com.mx',
'icloud.com':'Others',
'msn.com':'Others',
'live.com':'Others',
'live.com.mx':'Others',
'verizon.net':'Others',
'me.com':'Others',
'sbcglobal.net':'Others',
'cox.net':'Others',
'outlook.es':'Others',
'att.net':'Others',
'bellsouth.net':'Others',
'hotmail.fr':'Others',
'hotmail.es':'Others',
'web.de':'Others',
'ymail.com':'Others',
'prodigy.net.mx':'Others',
'mac.com':'Others',
'optonline.net':'Others',
'gmx.de':'Others',
'yahoo.fr':'Others',
'mail.com':'Others',
'charter.net':'Others',
'hotmail.co.uk':'Others',
'yahoo.de':'Others',
'gmail':'Others',
'embarqmail.com':'Others',
'earthlink.net':'Others',
'rocketmail.com':'Others',
'yahoo.es':'Others',
'scranton.edu':'Others',
'roadrunner.com':'Others',
'frontier.com':'Others',
'windstream.net':'Others',
'live.fr':'Others',
'juno.com':'Others',
'protonmail.com':'Others',
'hotmail.de':'Others',
'yahoo.co.uk':'Others',
'servicios-ta.com':'Others',
'yahoo.co.jp':'Others',
'aim.com':'Others',
'cfl.rr.com':'Others',
'suddenlink.net':'Others',
'ptd.net':'Others',
'q.com':'Others',
'twc.com':'Others',
'cableone.net':'Others',
'centurylink.net':'Others',
'frontiernet.net':'Others',
'sc.rr.com':'Others',
'netzero.com':'Others',
'netzero.net':'Others'}}

In [46]:
replace_d_info = {'DeviceInfo':{'Missing':'Missing',
'Windows':'Windows',
'iOS Device':'iOS Device',
'MacOS':'MacOS',
'Trident/7.0':'Trident/7.0',
'rv:11.0':'rv:11.0',
'rv:57.0':'Others',
'SM-J700M Build/MMB29K':'Others',
'SM-G610M Build/MMB29K':'Others',
'SM-G531H Build/LMY48B':'Others',
'rv:59.0':'Others',
'SM-G955U Build/NRD90M':'Others',
'SM-G935F Build/NRD90M':'Others',
'SM-G532M Build/MMB29T':'Others',
'ALE-L23 Build/HuaweiALE-L23':'Others',
'SM-G950U Build/NRD90M':'Others',
'rv:58.0':'Others',
'SM-G930V Build/NRD90M':'Others',
'rv:52.0':'Others',
'SAMSUNG':'Others',
'SM-G950F Build/NRD90M':'Others',
'Moto G (4) Build/NPJ25.93-14.7':'Others',
'SM-A300H Build/LRX22G':'Others',
'hi6210sft Build/MRA58K':'Others',
'SM-G570M Build/MMB29K':'Others',
'SM-N950U Build/NMF26X':'Others',
'rv:48.0':'Others',
'SM-J320M Build/LMY47V':'Others',
'SM-J730GM Build/NRD90M':'Others',
'CAM-L03 Build/HUAWEICAM-L03':'Others',
'LG-M700 Build/NMF26X':'Others',
'SM-J510MN Build/MMB29M':'Others',
'LG-D693n Build/LRX22G':'Others',
'SM-J701M Build/NRD90M':'Others',
'Moto E (4) Plus Build/NMA26.42-69':'Others',
'SM-G935V Build/NRD90M':'Others',
'LG-K410 Build/LRX22G':'Others',
'SM-G930F Build/NRD90M':'Others',
'Moto Z2 Play Build/NPSS26.118-19-14':'Others',
'SM-G955F Build/NRD90M':'Others',
'SM-A520F Build/NRD90M':'Others',
'Moto G (4) Build/NPJS25.93-14-13':'Others',
'PRA-LX3 Build/HUAWEIPRA-LX3':'Others',
'Moto G Play Build/MPIS24.241-15.3-7':'Others',
'Moto E (4) Build/NMA26.42-69':'Others',
'TRT-L53 Build/HUAWEITRT-L53':'Others',
'HUAWEI TAG-L13 Build/HUAWEITAG-L13':'Others',
'Moto G (5) Plus Build/NPN25.137-92':'Others',
'SAMSUNG SM-J700M Build/MMB29K':'Others',
'HUAWEI VNS-L53 Build/HUAWEIVNS-L53':'Others',
'Moto X Play Build/NPD26.48-24-1':'Others',
'rv:56.0':'Others',
'SAMSUNG SM-G950U Build/NRD90M':'Others',
'SAMSUNG SM-G532M Build/MMB29T':'Others',
'SM-A720F Build/NRD90M':'Others',
'Moto G (4) Build/NPJS25.93-14-10':'Others',
'LG-K580 Build/MRA58K':'Others',
'WAS-LX3 Build/HUAWEIWAS-LX3':'Others',
'HUAWEI VNS-L23 Build/HUAWEIVNS-L23':'Others',
'SM-J700M Build/LMY48B':'Others',
'BLADE V8 SE Build/NRD90M':'Others',
'Linux x86_64':'Others',
'LG-X230 Build/MRA58K':'Others',
'SAMSUNG SM-G955U Build/NRD90M':'Others',
'Moto G (5) Build/NPP25.137-93':'Others',
'Moto E (4) Build/NMA26.42-19':'Others',
'LG-X180g Build/LMY47I':'Others',
'SM-G920V Build/NRD90M':'Others',
'HTC Desire 10 lifestyle Build/MMB29M':'Others',
'MotoG3 Build/MPIS24.65-33.1-2-16':'Others',
'F3213 Build/36.1.A.1.86':'Others',
'Moto G (4) Build/NPJ25.93-14.5':'Others',
'Moto C Build/NRD90M.063':'Others',
'XT1635-02 Build/NPN26.118-22-2':'Others',
'SM-J500M Build/LMY48B':'Others',
'Moto G (5) Build/NPP25.137-72':'Others',
'SAMSUNG SM-G935F Build/NRD90M':'Others',
'MYA-L13 Build/HUAWEIMYA-L13':'Others',
'LG-K200 Build/MXB48T':'Others',
'HUAWEI RIO-L03 Build/HUAWEIRIO-L03':'Others',
'LG-K500 Build/MMB29M':'Others',
'SM-G930T Build/NRD90M':'Others',
'SM-G900M Build/LRX21T':'Others',
'LG-K530 Build/MMB29M':'Others',
'LG-H870 Build/NRD90U':'Others',
'SM-G925I Build/NRD90M':'Others',
'HUAWEI G7-L03 Build/HuaweiG7-L03':'Others',
'CHC-U03 Build/HuaweiCHC-U03':'Others',
'SAMSUNG-SM-G930A Build/NRD90M':'Others',
'SM-J710MN Build/MMB29K':'Others',
'F3113 Build/33.3.A.1.97':'Others',
'SAMSUNG SM-G531H Build/LMY48B':'Others',
'Moto':'Others',
'Moto G (5) Plus Build/NPN25.137-72':'Others',
'LG-D320 Build/KOT49I.V10a':'Others',
'HTC Desire 626s Build/LMY47O':'Others',
'SM-G950U Build/R16NW':'Others',
'LG-X240 Build/MRA58K':'Others',
'Moto G (5) Build/NPPS25.137-15-11':'Others',
'CRO-L03 Build/HUAWEICRO-L03':'Others',
'RNE-L03 Build/HUAWEIRNE-L03':'Others',
'SAMSUNG SM-N950U Build/NMF26X':'Others',
'LG-H542 Build/MRA58K':'Others',
'SM-G900V Build/MMB29M':'Others',
'KFFOWI Build/LVY48F':'Others',
'SM-G530H Build/LRX22G':'Others',
'5010G Build/MRA58K':'Others',
'SM-G950F Build/R16NW':'Others',
'BLL-L23 Build/HUAWEIBLL-L23':'Others',
'LG-H840 Build/NRD90U':'Others',
'BAC-L03 Build/HUAWEIBAC-L03':'Others',
'LG-H500 Build/LRX21Y':'Others',
'SAMSUNG SM-J701M Build/NRD90M':'Others',
'SAMSUNG SM-G950F Build/NRD90M':'Others',
'Moto G (5) Plus Build/NPNS25.137-15-11':'Others',
'Blade V6 Plus Build/MRA58K':'Others',
'5080A Build/MRA58K':'Others',
'Redmi Note 4 Build/NRD90M':'Others',
'Hisense L675 Build/MRA58K':'Others',
'Blade A510 Build/MRA58K':'Others',
'SM-A510M Build/MMB29K':'Others',
'SM-N950U Build/R16NW':'Others',
'Hisense F23 Build/NRD90M':'Others',
'HUAWEI CUN-L03 Build/HUAWEICUN-L03':'Others',
'SM-A510M Build/NRD90M':'Others',
'rv:60.0':'Others',
'GT-I9060M Build/KTU84P':'Others',
'HTC Desire 530 Build/MMB29M':'Others',
'SM-N950F Build/NMF26X':'Others',
'SM-G955U Build/R16NW':'Others',
'Moto Z (2':'Others',
'LG-X220 Build/LMY47I':'Others',
'SM-N920V Build/NRD90M':'Others',
'M4 SS4456 Build/LMY47V':'Others',
'Moto G (5) Build/NPP25.137-82':'Others',
'SM-G920P Build/NRD90M':'Others',
'SLA-L03 Build/HUAWEISLA-L03':'Others',
'SM-A500M Build/KTU84P':'Others',
'HTC Desire 650 Build/MMB29M':'Others',
'F5121 Build/34.3.A.0.252':'Others',
'Moto G (5) Plus Build/NPN25.137-82':'Others',
'SM-G610M Build/NRD90M':'Others',
'F3313 Build/37.0.A.2.108':'Others',
'Ilium LT510 Build/MRA58K':'Others',
'SAMSUNG SM-G930F Build/NRD90M':'Others',
'SM-T560 Build/KTU84P':'Others',
'SAMSUNG SM-G610M Build/MMB29K':'Others',
'SM-G935T Build/NRD90M':'Others',
'Moto G (4) Build/NPJS25.93-14-15':'Others',
'SAMSUNG SM-J730GM Build/NRD90M':'Others',
'SM-G930P Build/NRD90M':'Others',
'EVA-L09 Build/HUAWEIEVA-L09':'Others',
'SAMSUNG-SM-G935A Build/NRD90M':'Others',
'Lenovo A2016b30 Build/MRA58K':'Others',
'Redmi 4X Build/N2G47H':'Others',
'LG-H650 Build/MRA58K':'Others',
'SM-G935P Build/NRD90M':'Others',
'LGMP260 Build/NRD90U':'Others',
'Windows NT 6.1':'Others',
'es-us':'Others',
'F5121 Build/34.3.A.0.238':'Others',
'SM-J105B Build/LMY47V':'Others',
'MDDRJS':'Others',
'XT1032 Build/LPBS23.13-56-2':'Others',
'8050G Build/LMY47I':'Others',
'ALE-L21 Build/HuaweiALE-L21':'Others',
'SM-J111M Build/LMY47V':'Others',
'Moto G Play Build/MPIS24.241-15.3-26':'Others',
'MOT-A6020l37 Build/LMY47V':'Others',
'LG-X165g Build/LRX21M':'Others',
'KFDOWI Build/LVY48F':'Others',
'Nexus':'Others',
'5025G Build/LMY47I':'Others',
'LG-M250 Build/NRD90U':'Others',
'LG-X210 Build/LMY47I':'Others',
'SAMSUNG SM-J120H Build/LMY47V':'Others',
'SAMSUNG SM-G891A Build/NRD90M':'Others',
'SM-G570M Build/NRD90M':'Others',
'5015A Build/LMY47I':'Others',
'Blade V6 Build/LRX22G':'Others',
'SM-G530H Build/KTU84P':'Others',
'SM-J327P Build/MMB29M':'Others',
'LG-D680 Build/KOT49I':'Others',
'G3313 Build/43.0.A.5.79':'Others',
'Lenovo K33b36 Build/MMB29M':'Others',
'Lenovo':'Others',
'Pixel':'Others',
'TA-1039 Build/NMF26F':'Others',
'Hisense U963 Build/MRA58K':'Others',
'SAMSUNG SM-A520F Build/NRD90M':'Others',
'XT1585 Build/NCK25.118-10.5':'Others',
'Android 5.1':'Others',
'VS5012 Build/NRD90M':'Others',
'Ilium X510 Build/MRA58K':'Others',
'SM-G920F Build/NRD90M':'Others',
'Moto G (5) Build/NPP25.137-38':'Others',
'SM-N920T Build/NRD90M':'Others',
'rv:45.0':'Others',
'Pixel Build/OPM1.171019.011':'Others',
'Z981 Build/MMB29M':'Others',
'SAMSUNG SM-N950F Build/NMF26X':'Others',
'Lenovo PB2-650Y Build/MRA58K':'Others',
'LGLS676 Build/MXB48T':'Others',
'Lenovo-A6020l36 Build/LMY47V':'Others',
'SM-G925V Build/NRD90M':'Others',
'VS425PP Build/LMY47V':'Others',
'rv:51.0':'Others',
'HUAWEI':'Others',
'5012G Build/MRA58K':'Others',
'SM-T710 Build/NRD90M':'Others',
'SAMSUNG SM-G930A Build/NRD90M':'Others',
'SAMSUNG SM-G950F Build/R16NW':'Others',
'SM-J500M Build/MMB29M':'Others',
'Hisense F20 Build/MMB29M':'Others',
'SM-G355M Build/KOT49H':'Others',
'VS995 Build/NRD90M':'Others',
'F3213 Build/36.0.A.2.146':'Others',
'M4 SS4453 Build/MMB29M':'Others',
'Moto G (5S':'Others',
'moto x4 Build/OPWS27.57-40-6':'Others',
'5056A Build/MMB29M':'Others',
'Blade L5 Build/LMY47I':'Others',
'LG-K220 Build/MXB48T':'Others',
'XT1650 Build/NCLS26.118-23-13-3':'Others',
'SM-J530GM Build/NRD90M':'Others',
'LG-M320 Build/NRD90U':'Others',
'Build/OPR6.170623.013':'Others',
'VS988 Build/NRD90U':'Others',
'MotoG3 Build/MPI24.65-33.1-2':'Others',
'E5606 Build/30.2.A.1.21':'Others',
'SM-G928G Build/NRD90M':'Others',
'F3313 Build/37.0.A.2.248':'Others',
'SM-G550T1 Build/MMB29K':'Others',
'SM-G928V Build/NRD90M':'Others',
'SM-T550 Build/NMF26X':'Others',
'SM-A310M Build/LMY47X':'Others',
'SM-N910V Build/MMB29M':'Others',
'VS987 Build/NRD90U':'Others',
'Moto C Plus Build/NRD90M.05.034':'Others',
'Ilium X710 Build/MRA58K':'Others',
'Blade V580 Build/LMY47D':'Others',
'XT1064 Build/MPB24.65-34-3':'Others',
'SAMSUNG SM-G935A Build/NRD90M':'Others',
'G3223 Build/42.0.A.4.101':'Others',
'LG-H840 Build/MMB29M':'Others',
'SM-J120H Build/LMY47V':'Others',
'SAMSUNG SM-G930P Build/NRD90M':'Others',
'SM-G950U1 Build/NRD90M':'Others',
'SAMSUNG-SM-G928A Build/NRD90M':'Others',
'SM-A320FL Build/NRD90M':'Others',
'LGLS775 Build/NRD90U':'Others',
'Z835 Build/NMF26V':'Others',
'TA-1038 Build/NMF26O':'Others',
'SM-G955F Build/R16NW':'Others',
'Blade V6 Max Build/MRA58K':'Others',
'SAMSUNG SM-G920P Build/NRD90M':'Others',
'SAMSUNG-SM-G891A Build/NRD90M':'Others',
'gxq6580_weg_l Build/LMY47I':'Others',
'SM-A710M Build/LMY47X':'Others',
'5011A Build/NRD90M':'Others',
'rv:31.0':'Others',
'SAMSUNG SM-G955F Build/NRD90M':'Others',
'rv:54.0':'Others',
'SAMSUNG SM-T580 Build/NRD90M':'Others',
'Moto G (5) Plus Build/NPNS25.137-92-8':'Others',
'HTC One A9s Build/MRA58K':'Others',
'LG-H320 Build/LRX21Y':'Others',
'SAMSUNG SM-T587P Build/NRD90M':'Others',
'MotoE2(4G-LTE':'Others',
'SM-T350 Build/NMF26X':'Others',
'SM-E500M Build/KTU84P':'Others',
'Y635-L03 Build/HuaweiY635-L03':'Others',
'SM-A310F Build/NRD90M':'Others',
'SM-A320Y Build/NRD90M':'Others',
'SM-P350 Build/MMB29M':'Others',
'KFGIWI Build/LVY48F':'Others',
'WAS-LX1A Build/HUAWEIWAS-LX1A':'Others',
'SM-G965U Build/R16NW':'Others',
'SM-T377V Build/NMF26X':'Others',
'SM-T810 Build/NRD90M':'Others',
'SM-G920I Build/NRD90M':'Others',
'SAMSUNG SM-A310F Build/NRD90M':'Others',
'Moto G (5) Plus Build/NPNS25.137-92-4':'Others',
'Ilium L910 Build/MRA58K':'Others',
'rv:47.0':'Others',
'MYA-L03 Build/HUAWEIMYA-L03':'Others',
'Lenovo K33b36 Build/NRD90N':'Others',
'SM-T113NU Build/KTU84P':'Others',
'E2306 Build/26.3.A.1.33':'Others',
'SAMSUNG SM-G930T Build/NRD90M':'Others',
'XT1254 Build/MCG24.251-5-5':'Others',
'SAMSUNG SM-G928G Build/NRD90M':'Others',
'Blade L2 Plus Build/KOT49H':'Others',
'Pixel XL Build/OPM1.171019.011':'Others',
'rv:55.0':'Others',
'LGMS210 Build/NRD90U':'Others',
'HUAWEI VNS-L31 Build/HUAWEIVNS-L31':'Others',
'F3111 Build/33.3.A.1.97':'Others',
'SM-G935F Build/MMB29K':'Others',
'SM-N920A Build/MMB29K':'Others',
'XT1635-02':'Others',
'Moto G Play Build/NPIS26.48-36-5':'Others',
'E6603 Build/32.4.A.1.54':'Others',
'5054S Build/LMY47V':'Others',
'TA-1039 Build/N2G47H':'Others',
'LG-M400 Build/NRD90U':'Others',
'STF-L09 Build/HUAWEISTF-L09':'Others',
'SAMSUNG SM-G950U Build/R16NW':'Others',
'HUAWEI NXT-L09 Build/HUAWEINXT-L09':'Others',
'SM-G900F Build/MMB29M':'Others',
'SM-G9600 Build/R16NW':'Others',
'F8331 Build/41.2.A.7.76':'Others',
'SAMSUNG SM-G570M Build/MMB29K':'Others',
'rv:50.0':'Others',
'SM-A720F Build/MMB29K':'Others',
'SM-T280 Build/LMY47V':'Others',
'SM-J727T1 Build/NRD90M':'Others',
'BLADE A520 Build/NRD90M':'Others',
'SM-J700T Build/NMF26X':'Others',
'SM-A530F Build/NMF26X':'Others',
'Ilium L1120 Build/NRD90M':'Others',
'SAMSUNG SM-G530H Build/LRX22G':'Others',
'C6743 Build/LMY47V':'Others',
'ILIUM':'Others',
'SM-T800 Build/MMB29K':'Others',
'SAMSUNG SM-J327T1 Build/NRD90M':'Others',
'LG-H542 Build/LRX22G':'Others',
'SM-J327T1 Build/NRD90M':'Others',
'SM-G928T Build/NRD90M':'Others',
'SM-G800F Build/MMB29K':'Others',
'Pixel 2 Build/OPM1.171019.011':'Others',
'Moto G Play Build/NPIS26.48-36-2':'Others',
'moto x4 Build/NPW26.83-18-2-0-4':'Others',
'BLADE L7 Build/MRA58K':'Others',
'SM-N920P Build/NRD90M':'Others',
'SAMSUNG SM-G550T1 Build/MMB29K':'Others',
'SAMSUNG SM-N950U Build/R16NW':'Others',
'SM-S727VL Build/MMB29M':'Others',
'SAMSUNG SM-J710MN Build/MMB29K':'Others',
'LG-K430 Build/MRA58K':'Others',
'FIG-LX3 Build/HUAWEIFIG-LX3':'Others',
'XT1053 Build/LPAS23.12-21.7-1':'Others',
'Redmi':'Others',
'SAMSUNG-SM-G920A Build/NRD90M':'Others',
'MHA-L09 Build/HUAWEIMHA-L09':'Others',
'SM-G920T Build/NRD90M':'Others',
'LG-D331 Build/LRX22G':'Others',
'SAMSUNG-SM-G530AZ Build/LMY48B':'Others',
'SM-A510M Build/LMY47X':'Others',
'LG-K240 Build/MXB48T':'Others',
'SAMSUNG SM-J320M Build/LMY47V':'Others',
'Lenovo TB3-710F Build/LRX21M':'Others',
'Mi A1 Build/OPR1.170623.026':'Others',
'KFSUWI Build/LVY48F':'Others',
'rv:38.0':'Others',
'HTC One M9 Build/NRD90M':'Others',
'SM-G925T Build/NRD90M':'Others',
'SM-N900W8 Build/LRX21V':'Others',
'SAMSUNG SM-G925F Build/NRD90M':'Others',
'SAMSUNG SM-J700M Build/LMY48B':'Others',
'SM-T230NU Build/KOT49H':'Others',
'SM-J700H Build/MMB29K':'Others',
'VTR-L09 Build/HUAWEIVTR-L09':'Others',
'SM-A520W Build/NRD90M':'Others',
'Alcatel_5098O Build/MMB29M':'Others',
'XT1650':'Others',
'E2306 Build/26.1.A.3.111':'Others',
'BLU':'Others',
'LGMP450 Build/NRD90U':'Others',
'SM-G850M Build/LRX22G':'Others',
'E5506 Build/29.2.A.0.166':'Others',
'SM-J320FN Build/LMY47V':'Others',
'SAMSUNG SM-G900M Build/LRX21T':'Others',
'D6603 Build/23.5.A.1.291':'Others',
'LG-H872 Build/NRD90U':'Others',
'rv:49.0':'Others',
'KFTHWI Build/KTU84M':'Others',
'LG-H420 Build/LRX21Y':'Others',
'SM-T580 Build/NRD90M':'Others',
'Z956 Build/MMB29M':'Others',
'FRD-L09 Build/HUAWEIFRD-L09':'Others',
'BLADE V8 Build/NRD90M':'Others',
'SM-A500M Build/LRX22G':'Others',
'C6906 Build/14.6.A.1.236':'Others',
'MotoG3-TE Build/MPDS24.65-33-1-30':'Others',
'SAMSUNG SM-T710 Build/NRD90M':'Others',
'SAMSUNG SM-J327A Build/NRD90M':'Others',
'M4 SS4458 Build/MMB29M':'Others',
'Moto Z2 Play Build/NPS26.74-16-1':'Others',
'Android 4.4.2':'Others',
'rv:53.0':'Others',
'5010S Build/MRA58K':'Others',
'SM-A510F Build/NRD90M':'Others',
'SAMSUNG-SM-G900A Build/LRX21T':'Others',
'HTC':'Others',
'Moto E (4) Plus Build/NMA26.42-142':'Others',
'Pixel 2 XL Build/OPM1.171019.011':'Others',
'SAMSUNG-SM-G900A Build/MMB29M':'Others',
'XT1650 Build/NPLS26.118-20-5-11':'Others',
'SM-T813 Build/NRD90M':'Others',
'SM-J110M Build/LMY48B':'Others',
'LG-TP260 Build/NRD90U':'Others',
'SM-T350 Build/MMB29M':'Others',
'4047G Build/NRD90M':'Others',
'Moto G (4) Build/NPJS25.93-14-8.1-4':'Others',
'Lenovo TB3-710I Build/LMY47I':'Others',
'Windows NT 6.2':'Others',
'M4 SS4451 Build/LMY47D':'Others',
'HTC One_M8 Build/MRA58K':'Others',
'LG-H918 Build/NRD90M':'Others',
'SM-J727VPP Build/NRD90M':'Others',
'TR10CS1 Build/JDQ39':'Others',
'SAMSUNG SM-G935T Build/NRD90M':'Others',
'moto x4 Build/OPW27.57-40':'Others',
'SGH-I337M Build/LRX22C':'Others',
'SM-J100VPP Build/LMY48B':'Others',
'SM-A710M Build/NRD90M':'Others',
'GT-P5210 Build/JDQ39':'Others',
'SM-P580 Build/NRD90M':'Others',
'SM-G800M Build/KOT49H':'Others',
'Microsoft':'Others',
'F5121 Build/34.3.A.0.228':'Others',
'Mi A1 Build/N2G47H':'Others',
'SAMSUNG SM-A510M Build/NRD90M':'Others',
'F5321 Build/34.3.A.0.238':'Others',
'LG-M210 Build/NRD90U':'Others',
'E5506 Build/29.1.A.0.101':'Others',
'Moto Z2 Play Build/NPSS26.118-19-6':'Others',
'BLADE A602 Build/MRA58K':'Others',
'SM-G550T Build/MMB29K':'Others',
'SAMSUNG SM-G610M Build/NRD90M':'Others',
'Touch':'Others',
'rv:41.0':'Others',
'SM-T530NU Build/LRX22G':'Others',
'Z982 Build/NMF26V':'Others',
'Hisense F102 Build/NRD90M':'Others',
'SAMSUNG SM-G955U Build/R16NW':'Others',
'SM-J200M Build/LMY47X':'Others',
'XT1710-02 Build/NDSS26.118-23-19-6':'Others',
'Linux i686':'Others',
'SM-G920V Build/MMB29K':'Others',
'SAMSUNG-SM-G925A Build/MMB29K':'Others',
'Ilium X520 Build/NRD90M':'Others',
'Moto E (4) Build/NDQS26.69-23-2-3':'Others',
'LG-H850 Build/MMB29M':'Others',
'SM-G9650 Build/R16NW':'Others',
'BLADE V8Q Build/N2G47H':'Others',
'SM-G900T Build/MMB29M':'Others',
'KFAUWI Build/LVY48F':'Others',
'Z813 Build/LMY47O':'Others',
'KFTBWI Build/LVY48F':'Others',
'Aquaris':'Others',
'moto x4 Build/NPW26.83-42':'Others',
'Android 5.1.1':'Others',
'Moto G (4) Build/NPJS25.93-14-18':'Others',
'XT1635-01 Build/NDNS26.118-23-12-3':'Others',
'SAMSUNG SM-G965U Build/R16NW':'Others',
'SM-G960U Build/R16NW':'Others',
'LG-K530 Build/NRD90U':'Others',
'G3123 Build/40.0.A.6.175':'Others',
'5049W Build/NRD90M':'Others',
'GT-S7580L Build/JDQ39':'Others',
'SM-J730G Build/NRD90M':'Others',
'SM-T827V Build/NRD90M':'Others',
'SAMSUNG SM-G900F Build/MMB29M':'Others',
'SAMSUNG SM-J510MN Build/MMB29M':'Others',
'K88 Build/MMB29M':'Others',
'Ilium LT500 Build/LMY47O':'Others',
'Lenovo YT3-850M Build/MMB29M':'Others',
'SAMSUNG-SM-T377A Build/MMB29K':'Others',
'SAMSUNG-SGH-I337 Build/KOT49H':'Others',
'SAMSUNG SM-A720F Build/NRD90M':'Others',
'SM-G900M Build/KOT49H':'Others',
'XT1063 Build/MPB24.65-34':'Others',
'4013M Build/KOT49H':'Others',
'SAMSUNG SM-A320FL Build/NRD90M':'Others',
'Pixel Build/OPR3.170623.013':'Others',
'D5803 Build/23.5.A.1.291':'Others',
'SM-A520F Build/MMB29K':'Others',
'XT1030 Build/SU6-7.7':'Others',
'ZTE':'Others',
'SM-T377P Build/NMF26X':'Others',
'SM-T713 Build/NRD90M':'Others',
'Moto C Build/NRD90M.057':'Others',
'SM-P355M Build/MMB29M':'Others',
'SM-J530F Build/NRD90M':'Others',
'SAMSUNG SM-J327T Build/NRD90M':'Others',
'HUAWEI Y625-U13 Build/HUAWEIY625-U13':'Others',
'LG-LS997 Build/NRD90M':'Others',
'SM-G930V Build/MMB29M':'Others',
'VKY-L09 Build/HUAWEIVKY-L09':'Others',
'HUAWEI Y560-L03 Build/HUAWEIY560-L03':'Others',
'SAMSUNG SM-N920T Build/NRD90M':'Others',
'Blade A465 Build/LMY47D':'Others',
'SM-P550 Build/NMF26X':'Others',
'Redmi 3S Build/MMB29M':'Others',
'SAMSUNG SM-G935F Build/MMB29K':'Others',
'Moto Z2 Play Build/NPSS26.118-19-11':'Others',
'Moto G (5) Build/NPPS25.137-93-4':'Others',
'LG-LS993 Build/NRD90U':'Others',
'SM-G610F Build/NRD90M':'Others',
'P008 Build/NRD90M':'Others',
'SM-J327T Build/NRD90M':'Others',
'SAMSUNG SM-T810 Build/NRD90M':'Others',
'Ilium X210 Build/LMY47I':'Others',
'XT1650 Build/NCLS26.118-23-13-6-5':'Others',
'LG-D373 Build/KOT49I.V10a':'Others',
'Le X520 Build/IEXCNFN5902303111S':'Others',
'PRA-LX1 Build/HUAWEIPRA-LX1':'Others',
'SM-J727T Build/NRD90M':'Others',
'SM-T530 Build/LRX22G':'Others',
'Blade L3 Build/KOT49H':'Others',
'LG-D855 Build/LRX21R':'Others',
'SM-T700 Build/MMB29K':'Others',
'SAMSUNG SM-J250M Build/NMF26X':'Others',
'D5316 Build/19.4.A.0.182':'Others',
'Pixel 2 Build/OPM1.171019.021':'Others',
'SAMSUNG-SM-J327A Build/NRD90M':'Others',
'M4 SS4457 Build/MRA58K':'Others',
'SM-G965F Build/R16NW':'Others',
'SM-G900P Build/MMB29M':'Others',
'G3223 Build/42.0.A.4.167':'Others',
'SM-J700T1 Build/NMF26X':'Others',
'SM-N900T Build/LRX21V':'Others',
'SM-G930U Build/NRD90M':'Others',
'LG-K450 Build/MXB48T':'Others',
'LG-TP450 Build/NRD90U':'Others',
'SM-J327V Build/NRD90M':'Others',
'Redmi 5 Plus Build/N2G47H':'Others',
'LenovoA3300-GV Build/JDQ39':'Others',
'SM-T230 Build/KOT49H':'Others',
'SM-A730F Build/NMF26X':'Others',
'SM-N910F Build/MMB29M':'Others',
'SM-G900M Build/MMB29M':'Others',
'KFTT Build/IML74K':'Others',
'SM-G930W8 Build/NRD90M':'Others',
'XT1563 Build/MPD24.107-52':'Others',
'SAMSUNG SM-G930R4 Build/NRD90M':'Others',
'SM-T560NU Build/NMF26X':'Others',
'ALP-L09 Build/HUAWEIALP-L09':'Others',
'G3123':'Others',
'Ilium Pad T7X Build/LMY47I':'Others',
'HUAWEI GRA-L09 Build/HUAWEIGRA-L09':'Others',
'Build/OPM1.171019.011':'Others',
'SOV33 Build/35.0.D.2.25':'Others',
'Redmi 4A Build/MMB29M':'Others',
'SAMSUNG SM-G935P Build/NRD90M':'Others',
'EVA-L19 Build/HUAWEIEVA-L19':'Others',
'SAMSUNG SM-G892A Build/NRD90M':'Others',
'rv:43.0':'Others',
'SAMSUNG SM-A300H Build/LRX22G':'Others',
'LG-D331':'Others',
'SM-G730W8 Build/KOT49H':'Others',
'SM-N920G Build/NRD90M':'Others',
'5085B Build/MRA58K':'Others',
'SM-T800':'Others',
'SM-J710MN Build/NRD90M':'Others',
'Pixel 2 XL Build/OPM2.171019.029':'Others',
'XT1021 Build/LPCS23.13-34.8-3':'Others',
'GT-I9060L Build/JDQ39':'Others',
'SAMSUNG SM-J700T Build/NMF26X':'Others',
'SAMSUNG SM-J530F Build/NRD90M':'Others',
'M4':'Others',
'Tmovi Build/Vision':'Others',
'SAMSUNG SM-N920P Build/NRD90M':'Others',
'QTAQZ3 Build/LMY47V':'Others',
'SAMSUNG SM-N910A Build/MMB29M':'Others',
'ALP-L09 Build/HUAWEIALP-L09S':'Others',
'SAMSUNG SM-G890A Build/NRD90M':'Others',
'Ilium L610 Build/MRA58K':'Others',
'SAMSUNG SM-N950U1 Build/NMF26X':'Others',
'SM-G930F Build/MMB29K':'Others',
'LG-LS777 Build/NRD90U':'Others',
'Pixel Build/OPM4.171019.016.B1':'Others',
'Moto Z2 Play Build/NPSS26.118-19-4':'Others',
'FRD-L04 Build/HUAWEIFRD-L04':'Others',
'Moto G (5) Plus Build/NPNS25.137-92-10':'Others',
'LG-H910 Build/NRD90M':'Others',
'LGMS550 Build/NRD90U':'Others',
'RNE-L21 Build/HUAWEIRNE-L21':'Others',
'QTASUN1 Build/NRD90M':'Others',
'SM-J727V Build/NRD90M':'Others',
'F8331':'Others',
'ZTE Blade L5 Build/LMY47I':'Others',
'9003A Build/MRA58K':'Others',
'SAMSUNG-SM-N920A Build/NRD90M':'Others',
'moto x4 Build/OPWS27.57-40-14':'Others',
'LGMS330 Build/LMY47V':'Others',
'NOKIA':'Others',
'BLU STUDIO C 5+5 Build/LRX21M':'Others',
'F5121 Build/34.4.A.2.19':'Others',
'Nexus 6 Build/MOB30M':'Others',
'SAMSUNG SM-N920A Build/NRD90M':'Others',
'SAMSUNG SM-T530NU Build/LRX22G':'Others',
'F3113 Build/33.2.A.4.70':'Others',
'rv:39.0':'Others',
'XT1021 Build/KXC21.5-40':'Others',
'RCT6203W46 Build/KOT49H':'Others',
'XT1563 Build/MPDS24.107-52-5':'Others',
'SAMSUNG SM-J727T1 Build/NRD90M':'Others',
'LG-H815 Build/MRA58K':'Others',
'SM-N920C Build/NRD90M':'Others',
'ZTE Blade A511 Build/MRA58K':'Others',
'SAMSUNG SM-J500M Build/MMB29M':'Others',
'Ilium LT520 Build/NRD90M':'Others',
'SAMSUNG SM-J320FN Build/LMY47V':'Others',
'SM-J710GN Build/MMB29K':'Others',
'GT-N5110 Build/JDQ39':'Others',
'Studio':'Others',
'LGMS550 Build/MXB48T':'Others',
'KFAPWI Build/KTU84M':'Others',
'SM-J727U Build/NRD90M':'Others',
'STV100-4 Build/LMY47V':'Others',
'BLN-L21 Build/HONORBLN-L21':'Others',
'Moto G (5) Build/NPP25.137-33':'Others',
'F3113 Build/33.2.A.3.81':'Others',
'ALCATEL':'Others',
'SM-G925T Build/LMY47X':'Others',
'LG-V410':'Others',
'ALCATEL ONE TOUCH 7040A Build/JDQ39':'Others',
'MotoE2 Build/LPCS23.13-56-5':'Others',
'BLA-L29 Build/HUAWEIBLA-L29':'Others',
'rv:46.0':'Others',
'8062 Build/MRA58K':'Others',
'SAMSUNG SM-S727VL Build/MMB29M':'Others',
'XT1565':'Others',
'Pixel Build/OPM2.171019.029':'Others',
'ALCATEL ONE TOUCH 7047A Build/JDQ39':'Others',
'Lenovo A6020l37 Build/LMY47V':'Others',
'LG-H811 Build/MRA58K':'Others',
'Z832 Build/MMB29M':'Others',
'XT1040':'Others',
'rv:35.0':'Others',
'AX920':'Others',
'SAMSUNG SM-G920F Build/LRX22G':'Others',
'LG-M327 Build/NRD90U':'Others',
'MotoG3 Build/MPI24.65-25':'Others',
'XT1063 Build/MPB24.65-34-3':'Others',
'XT1650 Build/NPLS26.118-20-5-3':'Others',
'SM-G903F Build/MMB29K':'Others',
'E6553':'Others',
'SM-N910C Build/MMB29K':'Others',
'E2104 Build/24.0.A.5.14':'Others',
'VS501 Build/NRD90U':'Others',
'XT1635-01':'Others',
'Moto C Build/NRD90M.070':'Others',
'LG-H990 Build/NRD90M':'Others',
'ONEPLUS A5000 Build/NMF26X':'Others',
'VS986 Build/MRA58K':'Others',
'ASUS_X008DC Build/NRD90M':'Others',
'E5803 Build/32.4.A.1.54':'Others',
'D5306 Build/19.4.A.0.182':'Others',
'Android':'Others',
'Nexus 6P Build/OPR5.170623.011':'Others',
'LG-D850':'Others',
'TREKKER-X3 Build/MMB29M':'Others',
'XT1710-02 Build/NDSS26.118-23-15':'Others',
'E5306 Build/27.3.A.0.165':'Others',
'Moto G Play Build/NPI26.48-36':'Others',
'SCH-I545 Build/LRX22C':'Others',
'Moto G (4) Build/NPJ25.93-14':'Others',
'Pixel Build/OPM1.171019.016':'Others',
'BAH-L09 Build/HUAWEIBAH-L09':'Others',
'SM-G955U1 Build/NRD90M':'Others',
'SM-T113 Build/KTU84P':'Others',
'SM-G800F Build/KOT49H':'Others',
'Moto G (5) Plus Build/NPN25.137-83':'Others',
'SM-J710F Build/NRD90M':'Others',
'SM-G360T1 Build/LMY47X':'Others',
'SAMSUNG-SM-G890A Build/NRD90M':'Others',
'moto':'Others',
'LG-M710 Build/NRD90U':'Others',
'TA-1028 Build/NRD90M':'Others',
'SAMSUNG SM-G965F Build/R16NW':'Others',
'SAMSUNG SM-G925I Build/NRD90M':'Others',
'2PS64 Build/NRD90M':'Others',
'SAMSUNG SM-A320Y Build/NRD90M':'Others',
'SAMSUNG-SM-J320A Build/MMB29K':'Others',
'XT1032 Build/LPBS23.13-57-2':'Others',
'Z983 Build/NMF26F':'Others',
'E6853 Build/32.4.A.1.54':'Others',
'Android 7.0':'Others',
'Redmi Note 4X Build/MRA58K':'Others',
'VTR-AL00 Build/HUAWEIVTR-AL00':'Others',
'SM-P550 Build/MMB29M':'Others',
'Nexus 6 Build/N6F27M':'Others',
'Turbo C5 Build/LMY47I':'Others',
'SAMSUNG SM-A510F Build/NRD90M':'Others',
'SAMSUNG-SM-G925A Build/NRD90M':'Others',
'TA-1028 Build/NMF26O':'Others',
'G3223':'Others',
'Hisense':'Others',
'E5306 Build/27.3.A.0.129':'Others',
'Ilium L950 Build/KTU84P':'Others',
'BLA-L09 Build/HUAWEIBLA-L09':'Others',
'MotoG3-TE Build/MPD24.65-33':'Others',
'SM-T820 Build/NRD90M':'Others',
'F5321':'Others',
'MI':'Others',
'Android 6.0.1':'Others',
'M4 SS4450 Build/MRA58K':'Others',
'SAMSUNG-SM-G920A Build/MMB29K':'Others',
'LGUS215 Build/NRD90U':'Others',
'Alcatel_4060O Build/MMB29M':'Others',
'XT1021':'Others',
'MHA-L29 Build/HUAWEIMHA-L29':'Others',
'SAMSUNG SM-G920F Build/NRD90M':'Others',
'ONE':'Others',
'SM-G850M':'Others',
'ALCATEL ONE TOUCH 5036A Build/JDQ39':'Others',
'Lenovo PB1-750M Build/S100':'Others',
'MotoG3':'Others',
'SM-G950W':'Others',
'LM-X210(G':'Others',
'C6906':'Others',
'SM-N910T Build/MMB29M':'Others',
'LG-H830 Build/NRD90U':'Others',
'LGL164VL Build/NRD90U':'Others',
'Aquaris V Build/N2G47H':'Others',
'HTC One A9 Build/NRD90M':'Others',
'Nexus 5X Build/OPM3.171019.013':'Others',
'SM-A510F Build/MMB29K':'Others',
'SAMSUNG SM-G925T Build/NRD90M':'Others',
'XT1097':'Others',
'SM-J500FN Build/MMB29M':'Others',
'Vivo':'Others',
'SAMSUNG SM-J500FN Build/MMB29M':'Others',
'Pixel XL Build/OPM1.171019.021':'Others',
'SM-T330NU Build/LMY47X':'Others',
'SM-G720AX':'Others',
'GT-I8190L Build/JZO54K':'Others',
'Nexus 6P Build/OPR5.170623.014':'Others',
'SAMSUNG SM-G900T Build/MMB29M':'Others',
'Z965 Build/NMF26V':'Others',
'G3313 Build/43.0.A.7.25':'Others',
'Pixel 2 XL Build/OPM2.171019.029.B1':'Others',
'SM-G928G Build/MMB29K':'Others',
'vivo':'Others',
'PULP 4G Build/LMY47V':'Others',
'MotoG3-TE Build/MPDS24.65-33-1-3':'Others',
'SM-T817V Build/NRD90M':'Others',
'SM-A310M Build/NRD90M':'Others',
'GT-P5210 Build/KOT49H':'Others',
'QTAIR7 Build/LMY47D':'Others',
'LG-M153 Build/MXB48T':'Others',
'Neffos C5 Build/LMY47D':'Others',
'SAMSUNG SM-G930F/XXU2DRC1 Build/NRD90M':'Others',
'SM-N950U1 Build/NMF26X':'Others',
'Moto G (5) Build/NPPS25.137-93-8':'Others',
'XT1080 Build/SU6-7.7':'Others',
'HUAWEI Y360-U23 Build/HUAWEIY360-U23':'Others',
'BLN-L24 Build/HONORBLN-L24':'Others',
'5054N':'Others',
'SAMSUNG SM-G570M Build/NRD90M':'Others',
'SM-T550':'Others',
'LG-H831 Build/NRD90U':'Others',
'HUAWEI CAN-L11 Build/HUAWEICAN-L11':'Others',
'Ilium L620 Build/NRD90M':'Others',
'SAMSUNG SM-G960U Build/R16NW':'Others',
'SAMSUNG SM-A510M Build/MMB29K':'Others',
'VTR-L29':'Others',
'Ilium':'Others',
'HTC 10 Build/NRD90M':'Others',
'P5046A':'Others',
'SAMSUNG SM-G920A Build/NRD90M':'Others',
'Moto Z2 Play Build/NPS26.118-19':'Others',
'ZTE BLADE V8 MINI Build/NRD90M':'Others',
'G3123 Build/40.0.A.6.135':'Others',
'XT1563 Build/MPD24.65-25':'Others',
'Moto G (4) Build/NPJS25.93-14-8':'Others',
'ASUS_Z017D':'Others',
'G3423':'Others',
'ALCATEL ONE TOUCH 7042A Build/JDQ39':'Others',
'Lenovo YT3-850F Build/MMB29M':'Others',
'E6853':'Others',
'TA-1044 Build/OPR1.170623.026':'Others',
'SM-G610F':'Others',
'TA-1027 Build/N2G47H':'Others',
'Moto C Plus Build/NRD90M.05.022':'Others',
'Moto G (5) Build/NPP25.137-15':'Others',
'H1711 Build/HUAWEIH1711':'Others',
'TA-1052':'Others',
'STV100-2 Build/MMB29M':'Others',
'Moto E (4) Build/NMA26.42-11-3':'Others',
'Pixel 2 XL Build/OPM1.171019.021':'Others',
'ONE A2005 Build/MMB29M':'Others',
'FRD-L14 Build/HUAWEIFRD-L14':'Others',
'E5823 Build/32.4.A.1.54':'Others',
'G8141':'Others',
'XT1032':'Others',
'SAMSUNG SM-G935W8 Build/NRD90M':'Others',
'ASUS_Z00AD Build/LRX21V':'Others',
'RS988 Build/MMB29M':'Others',
'LG-H820 Build/NRD90U':'Others',
'HTC Desire 510 Build/KOT49H':'Others',
'LG-H871':'Others',
'Redmi Note 3 Build/MMB29M':'Others',
'en-gb':'Others',
'HUAWEI CAN-L01 Build/HUAWEICAN-L01':'Others',
'F1f Build/LMY47V':'Others',
'Moto C Build/NRD90M.054':'Others',
'Z9 PLUS Build/NRD90M':'Others',
'Moto E (4) Build/NCQ26.69-56':'Others',
'Win64':'Others',
'LG-H650 Build/LMY47V':'Others',
'Pixel 2 Build/OPM1.171019.019':'Others',
'SAMSUNG SM-G925P Build/MMB29K':'Others',
'SM-G360V Build/LMY48B':'Others',
'SM-T310 Build/KOT49H':'Others',
'HTC Desire 526G Build/KOT49H':'Others',
'SAMSUNG SM-G920I Build/NRD90M':'Others',
'Pixel XL Build/OPM1.171019.016':'Others',
'LG-D855':'Others',
'AERIAL':'Others',
'SAMSUNG SM-A710M Build/NRD90M':'Others',
'SAMSUNG SM-G903F Build/MMB29K':'Others',
'Android 6.0':'Others',
'SM-G903M':'Others',
'Blade':'Others',
'Moto G (5) Plus Build/NPNS25.137-93-8':'Others',
'rv:42.0':'Others',
'SAMSUNG-SM-G870A Build/MMB29M':'Others',
'P4526A Build/NRD90M':'Others',
'SC-02H':'Others',
'Dream':'Others',
'E5506':'Others',
'GT-I9505 Build/LRX22C':'Others',
'5051A Build/MMB29M':'Others',
'SAMSUNG SM-G900A Build/MMB29M':'Others',
'SM-N950U1 Build/R16NW':'Others',
'LG-K428 Build/NRD90U':'Others',
'SM-T110 Build/JDQ39':'Others',
'ONE TOUCH 4016A Build/JDQ39':'Others',
'SM-J700T Build/MMB29K':'Others',
'LG-D693n':'Others',
'7055A Build/KVT49L':'Others',
'SM-T560NU Build/MMB29M':'Others',
'8080 Build/LRX21M':'Others',
'SAMSUNG-SM-G890A Build/MMB29K':'Others',
'Intel':'Others',
'Moto E (4) Plus Build/NMA26.42-11-3':'Others',
'S57 Build/KTU84P':'Others',
'LG-H631':'Others',
'SAMSUNG-SGH-I497':'Others',
'Z798BL Build/MMB29M':'Others',
'VS990 Build/MRA58K':'Others',
'VK700 Build/LRX22G':'Others',
'SM-G960F Build/R16NW':'Others',
'VS980 4G Build/LRX22G':'Others',
'Hisense F32 Build/NMF26F':'Others',
'SGH-M919 Build/KTU84P':'Others',
'Moto E (4) Plus Build/NMA26.42-152':'Others',
'STV100-1':'Others',
'Lenovo PB2-670Y Build/MRA58K':'Others',
'LG-H700 Build/NRD90U':'Others',
'X3402':'Others',
'SAMSUNG SM-G892A Build/R16NW':'Others',
'SAMSUNG-SM-G930A':'Others',
'SAMSUNG SM-P580 Build/NRD90M':'Others',
'HUAWEI VNS-L21 Build/HUAWEIVNS-L21':'Others',
'9008A Build/NRD90M':'Others',
'HTCD100LVWPP':'Others',
'moto g(6) play Build/OPP27.61-14-4':'Others',
'LG-K428 Build/MMB29M':'Others',
'SM-G920I':'Others',
'LG-K540 Build/MMB29M':'Others',
'LG-H901':'Others',
'ASUS_X015D Build/NRD90M':'Others',
'SM-N920V':'Others',
'SM-J510FN':'Others',
'LG-K550':'Others',
'F3311':'Others',
'Aquaris_A4.5':'Others',
'Lenovo TAB 2 A7-30GC Build/KOT49H':'Others',
'A1601 Build/LMY47I':'Others',
'SM-G930R4 Build/NRD90M':'Others',
'SM-A310F Build/MMB29K':'Others',
'Pixel XL Build/OPM4.171019.016.B1':'Others',
'E2104':'Others',
'4047A Build/NRD90M':'Others',
'SM-G920F Build/MMB29K':'Others',
'AX820 Build/MRA58K':'Others',
'A96 Build/LMY47I':'Others',
'A574BL Build/NMF26F':'Others',
'Moto E (4) Build/NDQS26.69-64-2':'Others',
'SM-G955U1 Build/R16NW':'Others',
'7048A Build/LRX22G':'Others',
'MotoG3 Build/MPIS24.107-55-2-17':'Others',
'ZUUM_ZEN_I Build/LRX21M':'Others',
'SM-P900':'Others',
'LG-K420':'Others',
'G3313':'Others',
'SM-G900P Build/LRX21T':'Others',
'KFJWI':'Others',
'BLADE V7 Build/MRA58K':'Others',
'iris 820 Build/MRA58K':'Others',
'Fractal':'Others',
'QTAQZ3 Build/KOT49H':'Others',
'TA-1027':'Others',
'TOMMY2':'Others',
'ONEPLUS A5010 Build/OPM1.171019.011':'Others',
'Pixel Build/OPM1.171019.012':'Others',
'E5823':'Others',
'orbis':'Others',
'GT-P5113':'Others',
'verykoolS5525':'Others',
'Techpad':'Others',
'XT1572 Build/NPHS25.200-15-8':'Others',
'VS425':'Others',
'TREKKER-M1':'Others',
'WAS-LX1 Build/HUAWEIWAS-LX1':'Others',
'SM-G318ML':'Others',
'HTC One Build/LRX22G':'Others',
'ALUMINI':'Others',
'en-us':'Others',
'M4 SS4452 Build/LMY47V':'Others',
'LG-V495':'Others',
'SAMSUNG SM-G920T Build/NRD90M':'Others',
'Lenovo YT3-X50F Build/MMB29M':'Others',
'LG-M430 Build/NRD90U':'Others',
'P5526A Build/NRD90M':'Others',
'HTC U11 Build/NMF26X':'Others',
'KIW-L24':'Others',
'SAMSUNG-SM-N900A Build/LRX21V':'Others',
'SAMSUNG-SM-N910A':'Others',
'LGLS770':'Others',
'E6810':'Others',
'PSPC550 Build/LMY47D':'Others',
'TA-1027 Build/OPR1.170623.026':'Others',
'Hisense L675 PRO Build/NRD90M':'Others',
'rv:44.0':'Others',
'SAMSUNG SM-G530T Build/LMY47X':'Others',
'SM-J727P':'Others',
'es-mx':'Others',
'SM-G390F':'Others',
'LG-K373':'Others',
'SM-G925I Build/MMB29K':'Others',
'SM-G920P':'Others',
'TA-1025':'Others',
'STK_Sync_5e':'Others',
'GT-I9500':'Others',
'SAMSUNG SM-J710MN Build/NRD90M':'Others',
'HTC6545LVW':'Others',
'SM-J320P Build/LMY47X':'Others',
'SM-J327VPP Build/NRD90M':'Others',
'SAMSUNG SM-J327P Build/MMB29M':'Others',
'SM-G920R4':'Others',
'LG-K371':'Others',
'NEM-L51 Build/HONORNEM-L51':'Others',
'SM-G928P Build/NRD90M':'Others',
'B1-750':'Others',
'SM-G928V':'Others',
'SM-G900F Build/KOT49H':'Others',
'E6810 Build/5.320VZ.03.r':'Others',
'KFSAWI':'Others',
'SGH-I317M':'Others',
'TA-1025 Build/NMF26F':'Others',
'Lenovo TB-7703X Build/S100':'Others',
'LGLS992':'Others',
'SM-S903VL':'Others',
'SAMSUNG SM-G850F/G850FXXS2CQD9 Build/LRX22G':'Others',
'SAMSUNG SM-J530GM Build/NRD90M':'Others',
'VS996':'Others',
'S.N.O.W.4':'Others',
'Lenovo TB-X103F Build/LenovoTB-X103F':'Others',
'0PM92':'Others',
'LT22i Build/6.2.A.1.100':'Others',
'SM-A320FL Build/MMB29K':'Others',
'Blade A475 Build/LMY47D':'Others',
'SM-T520':'Others',
'ZTE BLADE A6 Build/NMF26F':'Others',
'KFMEWI':'Others',
'GT-I9300':'Others',
'LG-K550 Build/NRD90U':'Others',
'XT1003':'Others',
'SM-P600 Build/LMY47X':'Others',
'SM-N900V Build/LRX21V':'Others',
'G3123 Build/40.0.A.6.189':'Others',
'SAMSUNG SM-G950U1 Build/NRD90M':'Others',
'HTC6535LVW':'Others',
'TA-1028 Build/O00623':'Others',
'LG-M257':'Others',
'SAMSUNG-SM-T337A Build/LMY47X':'Others',
'C1904':'Others',
'LGLS990':'Others',
'SM-T377V':'Others',
'rv:52.9':'Others',
'SM-G925F Build/NRD90M':'Others',
'SM-J100MU':'Others',
'Advance':'Others',
'LG-H830':'Others',
'LG-D722':'Others',
'SM-E700M':'Others',
'SM-P605V':'Others',
'XT1580':'Others',
'SAMSUNG SM-G9650 Build/R16NW':'Others',
'BLU ENERGY X PLUS Build/LRX21M':'Others',
'SM-J320V Build/NMF26X':'Others',
'LGMS631 Build/MRA58K':'Others',
'LG-LK460':'Others',
'AX821 Build/MRA58K':'Others',
'IdeaTab':'Others',
'Pixel 2 XL Build/OPM1.171019.013':'Others',
'verykools5004':'Others',
'SM-G900H Build/MMB29K':'Others',
'SM-E500M':'Others',
'SM-G900T1':'Others',
'SH-04F':'Others',
'SM-J530G':'Others',
'XT1032 Build/KXB21.14-L1.40':'Others',
'SAMSUNG-SM-J320A Build/NMF26X':'Others',
'P5006A':'Others',
'XT1033':'Others',
'A0001 Build/MHC19Q':'Others',
'HTCD160LVWPP':'Others',
'LGLS665 Build/LMY47V':'Others',
'LGMS428':'Others',
'SM-G900F Build/LRX21T':'Others',
'HUAWEI Y520-U03 Build/HUAWEIY520-U03':'Others',
'LG-D625':'Others',
'SM-G920W8':'Others',
'D2306':'Others',
'KFSOWI':'Others',
'LG-H900/H90022b':'Others',
'LG-H870DS Build/NRD90U':'Others',
'Z557BL':'Others',
'LGLS751':'Others',
'ONEPLUS':'Others',
'TAB7':'Others',
'TA-1044 Build/NMF26F':'Others',
'SGP621 Build/23.5.A.1.291':'Others',
'REX':'Others',
'MALC':'Others',
'SM-T670':'Others',
'VS835':'Others',
'D2306 Build/18.6.A.0.182':'Others',
'SM-N910T Build/LMY47X':'Others',
'PLUS':'Others',
'm3':'Others',
'Gravity Build/NRD90M':'Others',
'LG-H932':'Others',
'LGL52VL Build/LMY47V':'Others',
'SM-G928V Build/MMB29K':'Others',
'SM-J510FN Build/NMF26X':'Others',
'GT-N7100':'Others',
'SAMSUNG-SGH-I337':'Others',
'LGLS675 Build/LMY47V':'Others',
'LG-H815':'Others',
'Z971':'Others',
'RCT6773W22B':'Others',
'SM-J200H':'Others',
'N817':'Others',
'LG-V521':'Others',
'HTC One mini Build/KOT49H':'Others',
'NX521J':'Others',
'2PZC5':'Others',
'BOIE9':'Others',
'Redmi Note 4 Build/MMB29M':'Others',
'SM-G530T':'Others',
'6045I Build/LRX22G':'Others',
'SAMSUNG SM-T550 Build/NMF26X':'Others',
'Alcatel_5044R Build/NRD90M':'Others',
'MAMI':'Others',
'RCT6S03W12':'Others',
'MotoG3 Build/MPIS24.65-25.1-19':'Others',
'ZTE A2017U Build/NRD90M':'Others',
'SAMSUNG SM-N900T Build/LRX21V':'Others',
'EGO':'Others',
'E6633':'Others',
'KFASWI Build/LVY48F':'Others',
'D6503':'Others',
'DLI-L22 Build/HONORDLI-L22':'Others',
'LGL84VL Build/NRD90U':'Others',
'GT-I9506':'Others',
'SAMSUNG SM-A730F Build/NMF26X':'Others',
'5095I Build/MRA58K':'Others',
'SAMSUNG SM-N900W8 Build/LRX21V':'Others',
'LG-P714':'Others',
'GT-I9195L':'Others',
'LG-H810':'Others',
'SM-G530M':'Others',
'HTCD200LVW':'Others',
'SAMSUNG-SM-G900A Build/LMY47X':'Others',
'LG-K330':'Others',
'RCT6513W87 Build/MRA58K':'Others',
'CPH1723':'Others',
'SM-G386T':'Others',
'F80 PIABELLA Build/MRA58K':'Others',
'SAMSUNG SM-T813 Build/NRD90M':'Others',
'Lenovo TB2-X30F Build/LenovoTB2-X30F':'Others',
'SM-A300FU Build/MMB29M':'Others',
'LGLS991':'Others',
'VFD':'Others',
'D6708':'Others',
'SM-J700P':'Others',
'SM-G920V':'Others',
'BNTV400':'Others',
'XT1092':'Others',
'SGP521':'Others',
'LG-LS998':'Others',
'2PYB2':'Others',
'UL40':'Others',
'SAMSUNG SM-G9600 Build/R16NW':'Others',
'S60 Build/MMB29M':'Others',
'SM-J120FN Build/LMY47X':'Others',
'B3-A40':'Others',
'ASUS_X00ID':'Others',
'WAS-TL10 Build/HUAWEIWAS-TL10':'Others',
'XT890 Build/9.8.2I-50_SML-25':'Others',
'RCT6303W87M7 Build/MRA58K':'Others',
'ONE TOUCH 4033A Build/JDQ39':'Others',
'SM-G935U':'Others',
'REVVLPLUS':'Others',
'XT1580 Build/NPKS25.200-17-8':'Others',
'Alumini3Plus':'Others',
'SM-N900P Build/LRX21V':'Others',
'ZTE BLADE A512 Build/MMB29M':'Others',
'XT1609 Build/MPIS24.241-2.35-1-17':'Others',
'SAMSUNG SM-J500M Build/LMY48B':'Others',
'P00C':'Others',
'Aquaris X5 Plus Build/NMF26F':'Others',
'SM-G930VL Build/NRD90M':'Others',
'RS988':'Others',
'SAMSUNG-SM-G900A Build/KOT49H':'Others',
'SAMSUNG SM-G900I Build/MMB29M':'Others',
'COVET Build/NRD90M':'Others',
'MotoG3 Build/MPI24.65-25.1':'Others',
'LG-M200 Build/NRD90U':'Others',
'SM-G900W8':'Others',
'XT1575':'Others',
'SAMSUNG-SM-N900A':'Others',
'TRT-LX3':'Others',
'XT1068':'Others',
'ONE A2003 Build/MMB29M':'Others',
'SM-A300M Build/KTU84P':'Others',
'SM-G900R4':'Others',
'SLA-L23':'Others',
'G8141 Build/47.1.A.5.51':'Others',
'Z837VL':'Others',
'SAMSUNG-SM-J120A':'Others',
'BND-L34':'Others',
'SAMSUNG-SM-G935A':'Others',
'rv:14.0':'Others',
'XT1064':'Others',
'XT1575 Build/NPHS25.200-23-1':'Others',
'LG-P708g':'Others',
'SM-N9005 Build/LRX21V':'Others',
'TA-1025 Build/OPR1.170623.026':'Others',
'R1':'Others',
'SM-G925W8':'Others',
'F5121':'Others',
'SAMSUNG SM-A310F/A310FXXU4CRB1 Build/NRD90M':'Others',
'XT1008':'Others',
'Z839':'Others',
'Redmi 4A Build/N2G47H':'Others',
'4009F':'Others',
'5017A':'Others',
'Nexus 5 Build/M4B30Z':'Others',
'verykools4009':'Others',
'K90U':'Others',
'SAMSUNG SM-G925P Build/NRD90M':'Others',
'Build/OPR1.170623.032':'Others',
'LT30p':'Others',
'G527-U081':'Others',
'LG-H850':'Others',
'MI MAX 2 Build/NMF26F':'Others',
'SM-T377T':'Others',
'E5803':'Others',
'Mi':'Others',
'LG-E975':'Others',
'SM-T807V':'Others',
'LG-M150':'Others',
'AM508':'Others',
'SM-P607T':'Others',
'Star':'Others',
'HUAWEI M2-801W Build/HUAWEIM2-801W':'Others',
'Energy X 2 Build/E050L':'Others',
'SM-N910H':'Others',
'SGH-M919V':'Others',
'P5006A Build/NRD90M':'Others',
'C6903':'Others',
'SM-A320Y':'Others',
'Z410':'Others',
'XT1058 Build/LPAS23.12-21.7-1':'Others',
'XT1609 Build/NPIS26.48-38-3':'Others',
'LG-SP320':'Others',
'Moto C Build/NRD90M.046':'Others',
'iris 870 Build/MRA58K':'Others',
'DT0704K08':'Others',
'SM-G361F':'Others',
'Z963VL':'Others',
'ZTE-Z835':'Others',
'HUAWEI Build/MMB28B':'Others',
'TA-1032':'Others',
'C6603':'Others',
'SM-S550TL':'Others',
'SAMSUNG SM-G955F Build/R16NW':'Others',
'SM-G360V':'Others',
'K88':'Others',
'SM-J500H':'Others',
'SP7731G':'Others',
'LG-E980h':'Others',
'MTT':'Others',
'SM-J330FN Build/NRD90M':'Others',
'SAMSUNG-SM-N920A':'Others',
'Mobiistar_LAI_Yuna_X':'Others',
'iPhone':'Others',
'SM-T580':'Others',
'SAMSUNG SM-G965U1 Build/R16NW':'Others',
'Coolpad':'Others',
'rv:61.0':'Others',
'SAMSUNG SM-N950U1 Build/R16NW':'Others',
'ALTER':'Others',
'LG-K212':'Others',
'Edison':'Others',
'SAMSUNG SM-N910V Build/MMB29M':'Others',
'F8332':'Others',
'SM-T825 Build/NRD90M':'Others',
'ASTRO':'Others',
'CAM-L23':'Others',
'6037B':'Others',
'N9560 Build/NMF26F':'Others',
'4034G':'Others',
'LG-H960':'Others',
'iris80':'Others',
'XT1254':'Others',
'SAMSUNG SM-J700T1 Build/NMF26X':'Others',
'SPH-L720T':'Others',
'LG-LG870':'Others',
'A621R':'Others',
'NYX_A1':'Others',
'Pixel 2 Build/OPM2.171019.016':'Others',
'FTJ152D':'Others',
'LGLS775 Build/MMB29M':'Others',
'SAMSUNG-SM-T377A':'Others',
'A463BG':'Others',
'A1-850':'Others',
'GT-I9195I':'Others',
'LGL41C':'Others',
'Shift Build/LMY47I':'Others',
'LGL83BL':'Others',
'SM-N900T':'Others',
'LG-H540':'Others',
'LG-H221':'Others',
'SM-G360M':'Others',
'DUK-AL20':'Others',
'ANE-LX3 Build/HUAWEIANE-LX3':'Others',
'SM-G390W':'Others',
'NXA116QC164':'Others',
'SAMSUNG-SGH-I537':'Others',
'ZUR70016':'Others',
'LG-D213':'Others',
'LG-H520':'Others',
'SM-T237P':'Others',
'LGL58VL':'Others',
'WAS-LX2J':'Others',
'SM-G928G':'Others',
'BBB100-1':'Others',
'SM-S920L':'Others',
'GRACE':'Others',
'GRANT':'Others',
'F3113':'Others',
'PLAYTAB':'Others',
'GT-S7582':'Others',
'LG-V500':'Others',
'SM-T705':'Others',
'O1':'Others',
'SM-A500H':'Others',
'SM-G530FZ':'Others',
'Gigaset':'Others',
'F3111 Build/33.3.A.1.115':'Others',
'SM-J510F':'Others',
'G8142':'Others',
'HTC U11 Build/OPR6.170623.013':'Others',
'ASUS_X00DDA':'Others',
'Ilium X220 Build/MRA58K':'Others',
'ASUS_X00HD Build/NMF26F':'Others',
'LGMS345':'Others',
'ASUS_P00J':'Others',
'XT1563':'Others',
'LEX829':'Others',
'Dash':'Others',
'LG-H735 Build/LMY47V':'Others',
'SGH-M919N':'Others',
'Blade A460 Build/LMY47O':'Others',
'HELIO':'Others',
'HIGHWAY':'Others',
'RCT6K03W13':'Others',
'SGH-I337M':'Others',
'LG-V410/V41020c':'Others',
'Z2':'Others',
'ASUS_Z01BDC':'Others',
'SM-T807T':'Others',
'verykools5034':'Others',
'A3-A20':'Others',
'4027A Build/KOT49H':'Others',
'LGL57BL':'Others',
'SGH-I747M':'Others',
'SMART':'Others',
'NetHelper70':'Others',
'LG-H810/H81021z':'Others',
'SM-G903W':'Others',
'RNE-L22 Build/HUAWEIRNE-L22':'Others',
'SM-T377W':'Others',
'Redmi Note 5A Build/N2G47H':'Others',
'GT-P3100':'Others',
'GT-I8190N':'Others',
'BV7000':'Others',
'RNE-L23 Build/HUAWEIRNE-L23':'Others',
'Build/KOT49H':'Others',
'R8106':'Others',
'PE-TL10':'Others',
'Minion_Tab':'Others',
'SM-J710GN':'Others',
'S6000':'Others',
'SM-T585':'Others',
'9002A':'Others',
'SM-J7108':'Others',
'WOW64':'Others',
'LGLS755':'Others',
'SM-S902L':'Others',
'S70V':'Others',
'PH-1':'Others',
'E2303':'Others',
'ME173X':'Others',
'GT-P5110':'Others',
'BLU LIFE XL Build/L050U':'Others',
'LG-LS995':'Others',
'MYA-L23':'Others',
'SM-G388F':'Others',
'ASUS_X008D Build/NRD90M':'Others',
'FP2':'Others',
'SM-T285M':'Others',
'TA-1044':'Others',
'LS5':'Others',
'SM-A710M':'Others',
'rv:37.0':'Others',
'SM-G900FD':'Others',
'F5122':'Others',
'SM-G930P':'Others',
'rv:33.0':'Others',
'LG-H812':'Others',
'STUDIO_G_HD':'Others',
'SAMSUNG-SM-G920AZ':'Others',
'SM-G925I Build/LMY47X':'Others',
'E6653 Build/32.4.A.1.54':'Others',
'SAMSUNG-SM-G930AZ':'Others',
'ASUS_A001':'Others',
'NXA8QC116':'Others',
'BBA100-1':'Others',
'6039A Build/LRX22G':'Others',
'SM-G930R7':'Others',
'XT1096':'Others',
'X78':'Others',
'Alcatel':'Others',
'SM-G920T':'Others',
'IdeaTabA1000-F':'Others',
'Beat':'Others',
'Archos':'Others',
'GT-N8000':'Others',
'SAMSUNG-SM-G890A':'Others',
'LG-H820':'Others',
'LGMS323':'Others',
'SGP611':'Others',
'EML-L29 Build/HUAWEIEML-L29':'Others',
'DOMOS':'Others',
'SPH-L720':'Others',
'5056N':'Others',
'LG-H933':'Others',
'A952':'Others',
'SM-A520F Build/R16NW':'Others',
'SM-G928P':'Others',
'SM-T210':'Others',
'Alumini3 Build/MRA58K':'Others',
'VS500':'Others',
'SM-G615F':'Others',
'K10':'Others',
'M10/Q1010':'Others',
'Kylin':'Others',
'AX1060':'Others',
'LOGIC':'Others',
'CPH1701':'Others',
'LG-D680':'Others',
'KYOCERA-C6742A Build/LMY47V':'Others',
'LG-H525n Build/MRA58K':'Others',
'XT1609':'Others',
'SAMSUNG-SM-G925A':'Others',
'A3_mini':'Others',
'K92':'Others',
'SM-G386W':'Others',
'SAMSUNG SM-N920V Build/NRD90M':'Others',
'SM-G935T':'Others',
'SM-A500FU Build/MMB29M':'Others',
'VK810':'Others',
'GT-S7275B':'Others',
'SM-G730V':'Others',
'SM-P550':'Others',
'YOLO':'Others',
'Hisense E51 Build/LMY47V':'Others',
'SAMSUNG-SM-J727AZ':'Others',
'SM-G350M':'Others',
'rv:29.0':'Others',
'PRA507':'Others',
'SM-J330F':'Others',
'F8131':'Others',
'LG-E450f':'Others',
'SM-G550T2':'Others',
'PMID7102DC':'Others',
'XT1008 Build/LPBS23.13-56-2':'Others',
'SM-G920T1':'Others',
'XT1680':'Others',
'SM-G900I':'Others',
'ZEIA8':'Others',
'SM-G900L':'Others',
'SM-J320H':'Others',
'LGL62VL':'Others',
'SM-A310M':'Others',
'LG-K350':'Others',
'Robin':'Others',
'LG-H340AR':'Others',
'SM-T555':'Others',
'SM-G930T1':'Others',
'Y550-L02':'Others',
'H1611':'Others',
'Moto C Build/NRD90M.050':'Others',
'SM-N950W':'Others',
'SM-P355M':'Others',
'Le':'Others',
'A97':'Others',
'VK815':'Others',
'LIMIT':'Others',
'LG-H443/H44312g':'Others',
'SM-S120VL':'Others',
'Android 7.1.2':'Others',
'SM-J320W8':'Others',
'LG-M255':'Others',
'SM-T337V':'Others',
'A37f':'Others',
'LG-H931':'Others',
'Hisense T963 Build/MRA58K':'Others',
'SM-N920R4':'Others',
'5045I':'Others',
'LG-H343':'Others',
'E6790TM':'Others',
'Northwell':'Others',
'D2406':'Others',
'SM-G925I':'Others',
'XT1080':'Others',
'ZTE BLADE A321 Build/NMF26F':'Others',
'SM-G7105':'Others',
'SM-T320 Build/KOT49H':'Others',
'YOGA':'Others',
'SM-G530T1':'Others',
'E8QP':'Others',
'XT1225':'Others',
'SM-G930T':'Others',
'SKY_5.0LM':'Others',
'CLT-L09':'Others',
'SM-G925P Build/NRD90M':'Others',
'PLK-L01 Build/HONORPLK-L01':'Others',
'SAMSUNG-SM-G870A':'Others',
'HTC_One':'Others',
'Moto G (5) Plus Build/NPN25.137-15':'Others',
'BG2-W09':'Others',
'ATT-IE11':'Others',
'HUAWEI LUA-U23 Build/HUAWEILUA-U23':'Others',
'LG-H810/H81022f':'Others',
'AX1070':'Others',
'9022X':'Others',
'SM-T217S Build/KOT49H':'Others',
'SAMSUNG-SM-T677A':'Others',
'5057M':'Others',
'TA-1020':'Others',
'HT0703K16':'Others',
'D5316':'Others',
'G620S-L03':'Others',
'TA-1038 Build/O00623':'Others',
'BV8000Pro':'Others',
'E2006':'Others',
'LG-V496':'Others',
'SENS':'Others',
'B3-A20':'Others',
'WAS-L03T Build/HUAWEIWAS-L03T':'Others',
'SM-T537V':'Others',
'SM-G531F':'Others',
'XT1528':'Others',
'G630-U251':'Others',
'SM-G360F':'Others',
'XT1055':'Others',
'QMV7A':'Others',
'4003A':'Others',
'BV6000':'Others',
'Aquaris U Plus Build/NMF26F':'Others',
'D5503':'Others',
'SAMSUNG SM-J111M Build/LMY47V':'Others',
'SM-N910T3':'Others',
'R831L':'Others',
'VS985':'Others',
'SM-J320VPP':'Others',
'RCT6223W87':'Others',
'V502015':'Others',
'FEVER':'Others',
'NX16A8116KP':'Others',
'ASUS_Z00ED':'Others',
'E6683':'Others',
'LG-D320':'Others',
'E501':'Others',
'ASUS':'Others',
'E6603':'Others',
"F80'S+":'Others',
'1016S':'Others',
'GT-N5110':'Others',
'XT1635-02 Build/OPN27.76-12-22':'Others',
'Tab2A7-10F':'Others',
'LGL33L/V100':'Others',
'LG-D725':'Others',
'verykool':'Others',
'LG-V520':'Others',
'HTC_One_M8s/2.12.111.1':'Others',
'TA-1038':'Others',
'SAMSUNG SM-J320F Build/LMY47V':'Others',
'T08':'Others',
'GT-I9515':'Others',
'XT1032 Build/KXB20.9-1.10-1.24-1.1':'Others',
'SM-S320VL':'Others',
'TA-1004':'Others',
'XT1095':'Others',
'Moto G Play Build/NJH47F':'Others',
'SM-T377P':'Others',
'AERIAL Build/NRD90M':'Others',
'STV100-3':'Others',
'verykools5035':'Others',
'P5026A':'Others',
'SAMSUNG-SM-J327AZ':'Others',
'SM-G901F':'Others',
'D6603':'Others',
'XT1060':'Others',
'SM-N915V':'Others',
'SM-G530P':'Others',
'Fusion5_u7':'Others',
'SM-G928F':'Others',
'XT1580 Build/NPKS25.200-12-9':'Others',
'T1':'Others',
'XT1072':'Others',
'Z799VL':'Others',
'Z':'Others',
'LG-M154':'Others',
'SM-G955F':'Others',
'A50C+':'Others',
'H550':'Others',
'VS996 Build/OPR1.170623.026':'Others',
'LGUS991':'Others',
'SAMSUNG SM-J327AZ Build/NRD90M':'Others',
'SM-G955W':'Others',
'SM-T587P':'Others',
'Infinix':'Others',
'ASUS_Z01BDA':'Others',
'P00A':'Others',
'SAMSUNG-SM-J320AZ':'Others',
'verykoolS5530 Build/LMY47I':'Others',
'SM-J730F':'Others',
'SM-N910P':'Others',
'0PAJ5':'Others',
'SAMSUNG SM-T800 Build/MMB29K':'Others',
'Joy':'Others',
'AOC_U706G':'Others',
'Z812':'Others',
'LG-V522':'Others',
'ZA509':'Others',
'LG-V930':'Others',
'verykoolS5005':'Others',
'SM-G900V':'Others',
'XT1058':'Others',
'SAMSUNG-SM-G900A':'Others',
'Linux':'Others',
'iris50':'Others',
'VS880':'Others',
'SM-G313ML':'Others',
'LG-D681':'Others',
'NX785QC8G':'Others',
'G8341':'Others',
'AX921 Build/MRA58K':'Others',
'SGP511':'Others',
'A9':'Others',
'QTASUN1':'Others',
'XT1710-02':'Others',
'SM-N950F Build/R16NW':'Others',
'G25524K':'Others',
'Alcatel_4060A':'Others',
'P027':'Others',
'QwestIE8':'Others',
'HP':'Others',
'SAMSUNG-SM-T537A':'Others',
'BBB100-2':'Others',
'E6833':'Others',
'ASUS_Z01KD':'Others',
'SAMSUNG-SM-T817A':'Others',
'Swift':'Others',
'ME301T':'Others',
'Venue':'Others',
'STELLAR':'Others',
'M431':'Others',
'X900+':'Others',
'Grand':'Others',
'IdeaTabA2109A':'Others',
'A466BG':'Others',
'LG-M322':'Others',
'SM-G935V':'Others',
'ICON':'Others',
'ORION':'Others',
'LG-V700':'Others',
'Hisense F8 MINI Build/NRD90M':'Others'}}

In [47]:
X_train.replace(replace_p_email, inplace=True)

In [48]:
X_train.replace(replace_r_email, inplace=True)

In [49]:
X_train.replace(replace_d_info, inplace=True)

In [50]:
categorical_features = [feature for feature in X_train.columns if X_train[feature].dtypes == 'O']
print(categorical_features)

['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15', 'id_16', 'id_28', 'id_29', 'id_31', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']


In [51]:
from sklearn.preprocessing import MinMaxScaler

In [52]:
scaler = MinMaxScaler()

In [53]:
X_train[numerical_features] = scaler.fit_transform(X_train[numerical_features])

In [54]:
X_train[numerical_features].head()

,TransactionAmt,card1,card2,card3,card5,addr1,addr2,dist1,C1,C2,...,id_01,id_02,id_05,id_06,id_11,id_13,id_17,id_19,id_20,TransactionDT_derived
0,0.002137,0.743044,0.522,0.381679,0.306569,0.488636,0.836957,0.001847,0.000213,0.000176,...,0.95,0.125851,0.580645,1.0,1.0,0.777778,0.511628,0.422067,0.663102,0.000000e+00
1,0.000900,0.100885,0.608,0.381679,0.014599,0.511364,0.836957,0.000778,0.000213,0.000176,...,0.95,0.125851,0.580645,1.0,1.0,0.777778,0.511628,0.422067,0.663102,6.359409e-08
2,0.001840,0.210566,0.780,0.381679,0.481752,0.522727,0.836957,0.027902,0.000213,0.000176,...,0.95,0.125851,0.580645,1.0,1.0,0.777778,0.511628,0.422067,0.663102,4.387992e-06
3,0.001558,0.984824,0.934,0.381679,0.124088,0.854545,0.836957,0.000778,0.000427,0.000879,...,0.95,0.125851,0.580645,1.0,1.0,0.777778,0.511628,0.422067,0.663102,6.295815e-06
4,0.001558,0.201023,0.828,0.381679,0.014599,0.727273,0.836957,0.000778,0.000213,0.000176,...,1.00,0.070815,0.580645,1.0,1.0,0.777778,0.511628,0.774081,0.078431,6.740974e-06


In [56]:
X_train.shape

(590540, 358)

In [57]:
ProductCD=pd.get_dummies(X_train['ProductCD'],drop_first=True)
card4=pd.get_dummies(X_train['card4'],drop_first=True)
card6=pd.get_dummies(X_train['card6'],drop_first=True)
P_emaildomain=pd.get_dummies(X_train['P_emaildomain'],drop_first=True)
R_emaildomain=pd.get_dummies(X_train['R_emaildomain'],drop_first=True)
M1=pd.get_dummies(X_train['M1'],drop_first=True)
M2=pd.get_dummies(X_train['M2'],drop_first=True)
M3=pd.get_dummies(X_train['M3'],drop_first=True)
M4=pd.get_dummies(X_train['M4'],drop_first=True)
M5=pd.get_dummies(X_train['M5'],drop_first=True)
M6=pd.get_dummies(X_train['M6'],drop_first=True)
M7=pd.get_dummies(X_train['M7'],drop_first=True)
M8=pd.get_dummies(X_train['M8'],drop_first=True)
M9=pd.get_dummies(X_train['M9'],drop_first=True)
id_12=pd.get_dummies(X_train['id_12'],drop_first=True)
id_15=pd.get_dummies(X_train['id_15'],drop_first=True)
id_16=pd.get_dummies(X_train['id_16'],drop_first=True)
id_28=pd.get_dummies(X_train['id_28'],drop_first=True)
id_29=pd.get_dummies(X_train['id_29'],drop_first=True)
id_31=pd.get_dummies(X_train['id_31'],drop_first=True)
id_35=pd.get_dummies(X_train['id_35'],drop_first=True)
id_36=pd.get_dummies(X_train['id_36'],drop_first=True)
id_37=pd.get_dummies(X_train['id_37'],drop_first=True)
id_38=pd.get_dummies(X_train['id_38'],drop_first=True)
DeviceType=pd.get_dummies(X_train['DeviceType'],drop_first=True)
DeviceInfo=pd.get_dummies(X_train['DeviceInfo'],drop_first=True)

In [58]:
X_train.drop(['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1', 'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15', 'id_16', 'id_28', 'id_29', 'id_31', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo'],axis=1,inplace=True)

In [59]:
X_train.shape

(590540, 332)

In [60]:
X_train=pd.concat([X_train,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9,
                  id_12,id_15,id_16,id_28,id_29,id_31,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo],axis=1)

In [61]:
X_train.shape

(590540, 614)

### Join Target and save Data

In [62]:
to_model_data = pd.concat([X_train,y_train],axis=1)

In [63]:
to_model_data.columns

Index(['TransactionAmt', 'card1', 'card2', 'card3', 'card5', 'addr1', 'addr2',
       'dist1', 'C1', 'C2',
       ...
       'ZA990', 'ZTE-Z956', 'hp2015', 'iOS Device', 'iris702', 'rv:11.0',
       'verykoolS5019', 'verykoolS5524', 'xs-Z47b7VqTMxs', 'isFraud'],
      dtype='object', length=615)

In [64]:
to_model_data.shape

(590540, 615)

In [65]:
to_model_data.to_csv("./Output_CSVs/Prepped Data/iteration1_prepped_data.csv")

In [66]:
null_check_df = pd.DataFrame(to_model_data.isnull().sum())
null_check_df.to_csv("./Output_CSVs/Analysis Outputs/iteration1_null_check.csv")